In [ ]:
import numpy as np
import pickle
# !pip install bcolz
import bcolz

In [ ]:
from collections import Counter
from functools import reduce
# from tqdm.notebook import tqdm
import string
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
import torch
import urllib
import glob
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import nltk
from PIL import Image
from torchvision import transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet

In [ ]:
from PIL import Image
import pandas as pd
import spacy
from torch.nn.utils.rnn import pad_sequence
spacy_eng = spacy.load("en_core_web_sm")

In [ ]:
def read_img(file):
    img_id=[]
    for img in glob.glob(file+'*.jpg'):
        temp=os.path.basename(img)
        img_id.append(temp)
        
    return img_id